<a href="https://colab.research.google.com/github/skyprince999/100-Days-Of-ML/blob/master/Day_15_Class_Activations_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Class Activation Mapping in PyTorch: [Link](https://snappishproductions.com/blog/2018/01/03/class-activation-mapping-in-pytorch.html.html)

In [0]:
%matplotlib inline

from PIL import Image
from matplotlib.pyplot import imshow
from torchvision import models, transforms
from torch.autograd import Variable
from torch.nn import functional as F
from torch import topk
import numpy as np
import skimage.transform